<a href="https://colab.research.google.com/github/rgaguiar/recuperacao_endereco/blob/main/recuperar_enderecos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pandas geopy openpyxl

In [3]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

import warnings
warnings.filterwarnings("ignore")

In [4]:
# criando um token
geolocator = Nominatim(user_agent="my_geopy_app_v1.0")

# Usar o RateLimiter para evitar exceder os limites de requisições (um intervalo de 1 segundo entre cada requisição)
geocode = RateLimiter(geolocator.reverse, min_delay_seconds=1)

def get_bairro_rua_cidade_uf(latitude, longitude):
    try:
        location = geolocator.reverse((latitude, longitude), exactly_one=True)
        if location:
            address = location.raw['address']
            bairro = address.get('suburb', 'Bairro não encontrado')
            rua = address.get('road', 'Rua não encontrada')
            cidade = address.get('city', 'Cidade não encontrada')
            uf = address.get('state', 'UF não encontrada')
            return bairro, rua, cidade, uf
        else:
            return "Bairro não encontrado", "Rua não encontrada"
    except Exception as e:
        return f"Erro: {e}", f"Erro: {e}"

path = './table (26).csv'
dataset = pd.read_csv(path)

dataset = dataset.sample(5) # testando apenas com 5 elementos da base de dados

dataset['bairro'], dataset['rua'], dataset['cidade'], dataset['uf']  = zip(*dataset.apply(lambda row: get_bairro_rua_cidade_uf(row['Latitude'], row['Longitude']), axis=1))

dataset.to_excel('arquivo_com_bairros.xlsx', index=False)

display(dataset[["Latitude", "Longitude", "bairro", "rua", "cidade", "uf"]].head(10))


,Latitude,Longitude,bairro,rua,cidade,uf
1838,-3.844793,-38.645055,Jaçanaú,Rua Hermes Castelo Branco,Maracanaú,Ceará
2403,-3.796069,-38.468900,Sapiranga / Coité,Rua Tabelião Joaquim Coelho,Fortaleza,Ceará
4971,-3.668258,-38.675577,Icaraí,Rodovia Deputado Ted Rocha Pontes,Caucaia,Ceará
3765,-3.755250,-38.529236,Fátima,Avenida Eduardo Girão,Fortaleza,Ceará
274,-3.768302,-38.609681,Conjunto Ceará II,Rua 541 A,Fortaleza,Ceará
